In [45]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
# $example on$
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans
# $example off$


In [56]:
if sc:
    sc.stop()
    
sc = SparkContext().getOrCreate()  # SparkContext

In [57]:
from pyspark.mllib.clustering import StreamingKMeansModel

initCenters = [[0.0, 0.0], [1.0, 1.0]]
initWeights = [1.0, 1.0]
stkm = StreamingKMeansModel(initCenters, initWeights)
data = sc.parallelize([[-0.1, -0.1], [0.1, 0.1],
                       [0.9, 0.9], [1.1, 1.1], [1,1]])
stkm = stkm.update(data, 1.0, u"batches")
stkm.centers

array([[ 0.,  0.],
       [ 1.,  1.]])

In [58]:
stkm.predict([-0.1, -0.1])

0

In [59]:
stkm.predict([0.9, 0.9])

1

In [60]:
stkm.clusterWeights

[3.0, 4.0]

In [61]:
decayFactor = 0.0

In [62]:
data = sc.parallelize([[1.5, 1.5], [0.2, 0.2]])

In [63]:
stkm = stkm.update(data, 0.0, u"batches")

In [64]:
stkm.centers

array([[ 0.2,  0.2],
       [ 1.5,  1.5]])

In [65]:
stkm.clusterWeights

[1.0, 1.0]

In [66]:
stkm.predict([0.2, 0.2])

0

In [67]:
stkm.predict([1.5, 1.5])

1

In [68]:
data.collect()

[[1.5, 1.5], [0.2, 0.2]]